# Strongest Pokemon in each game

## Methodology

dolor ipsum

In [540]:
import urllib.request
import codecs
import re
import math
import numpy
import csv
import os.path

## Gen I

### Import data

In [542]:
if(not(os.path.isdir('gen_i_htmls'))):
    os.mkdir('gen_i_htmls')

In [552]:
url = "https://www.serebii.net/pokedex/"

urllib.request.urlretrieve(url, "gen_i_htmls/gen_i.txt")

outputFile = codecs.open("gen_i_htmls/gen_i.txt", "r", "ISO-8859-1")
filestring = outputFile.read()

dex = dict(re.findall("(?s)<option value=\"/pokedex/(.*?).shtml\">(.*?)</option>", filestring, re.IGNORECASE))

In [550]:
# Red and Blue Trades
url = 'https://www.serebii.net/rb/trades.shtml'
filename = 'gen_i_htmls/tradesrb.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
tradesourcerb = outputFile.read()


# Yellow Trades
url = 'https://www.serebii.net/yellow/trades.shtml'
filename = 'gen_i_htmls/tradesyellow.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
tradesourceyellow = outputFile.read()


# Red and Blue Gifts
url = 'https://www.serebii.net/rb/gift.shtml'
filename = 'gen_i_htmls/giftsrb.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
giftsourcerb = outputFile.read()

# Yellow Gifts
url = 'https://www.serebii.net/yellow/gift.shtml'
filename = 'gen_i_htmls/giftsyellow.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
giftsourceyellow = outputFile.read()

In [164]:
info = {}

for key in dex:
    url = "https://www.serebii.net/pokedex/" + key + ".shtml"
    file = "gen_i_htmls/" + dex[key] + ".txt"
    
    urllib.request.urlretrieve(url, file)
    
    outputFile = codecs.open(file, "r", "ISO-8859-1")
    info[key] = outputFile.read()

In [164]:
info = {}

for key in dex:
    url = "https://www.serebii.net/pokedex/" + key + ".shtml"
    file = "gen_i_htmls/" + dex[key] + ".txt"
    
    urllib.request.urlretrieve(url, file)
    
    outputFile = codecs.open(file, "r", "ISO-8859-1")
    info[key] = outputFile.read()

In [426]:
locations = {}

for key in dex:
    url = "https://www.serebii.net/pokedex/location/" + key + ".shtml#r"
    file = "gen_i_htmls/" + dex[key] + "_loc" +".txt"
    
    urllib.request.urlretrieve(url, file)
    
    outputFile = codecs.open(file, "r", "ISO-8859-1")
    locations[key] = outputFile.read()

### Parse Data

In [428]:
searchstring = '(?s)Base Stats - Total: (.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td></tr>\r\n'
stats = {}

for key in dex:
    stats[key] = [int(num) for num in re.findall(searchstring, info[key], re.IGNORECASE)[0]]

In [465]:
searchstring = '(?s)<img src="/pokearth/sprites/rb/(.*?).png" border="0" title="" />.*?Trader requests a <a href="/pokedex-dp/(.*?).shtml">'
tradesrb = re.findall(searchstring,tradesourcerb,re.IGNORECASE)

searchstring = '(?s)<img src="/pokearth/sprites/yellow/(.*?).png" border="0" title="" />.*?Trader requests a <a href="/pokedex-dp/(.*?).shtml">'
tradesyellow = re.findall(searchstring,tradesourceyellow,re.IGNORECASE)

In [501]:
searchstring = '(?s)a href="/pokedex/.*?.shtml"><img  src="/pokearth/sprites/rb/(.*?).png" border="0" title=".*?" /></a></td></tr><tr>.*?<td class="label">Level (.*?)</td>'
giftsrb = re.findall(searchstring,giftsourcerb,re.IGNORECASE)

searchstring = '(?s)a href="/pokedex/.*?.shtml"><img  src="/pokearth/sprites/yellow/(.*?).png" border="0" title=".*?" /></a></td></tr><tr>.*?<td class="label">Level (.*?)</td>'
giftsyellow = re.findall(searchstring,giftsourceyellow,re.IGNORECASE)

In [429]:
searchstring = 'not found in the wild'

wildpokemon = {key: dex[key] for key in dex if not(bool(re.search(searchstring, locations[key])))}

In [447]:
games = [key for key in locationstats['010']]

In [432]:
searchstring = '(?s)<table class="dextable"><tr > <td class="fooevo" colspan="6"><a name=".*?">(.*?)</a></td></tr>(.*?)</table>\r\n'

subsearchstring = '(?s)\t\t<a href="/pokearth/kanto/.*?.shtml">(.*?)</a>\t\t</td>\r\n\t\t<td class="fooinfo">(.*?)\t\t</td>\r\n\t\t<td class="fooinfo">(.*?)\t\t</td>\r\n\t\t<td class="fooinfo">(.*?)</td>\r\n\t\t<td class="fooinfo">(.*?)</td>'

locationstats = {}

for key in dex:
    locationstats[key] = {game[0]: re.findall(subsearchstring,game[1],re.IGNORECASE) for game in re.findall(searchstring, locations[key], re.IGNORECASE)}

In [527]:
trademinlevelsyellow = {'122': 8,\
                        '067': 16,\
                        '051': 15,\
                        '047': 13,\
                        '112': 15,\
                        '087': 15,\
                        '089': 25}

trademinlevelsrb = {'122': 7,\
                    '029': 2,\
                    '030': 16,\
                    '108': 15,\
                    '124': 15,\
                    '083': 2,\
                    '086': 28,\
                    '101': 3,\
                    '114': 13}

def get_max_level(pkmn,game):
    try:
        wildmax = max([int(spot[-1]) for spot in locationstats[pkmn][game] if spot[-1].isdigit()])
    except:
        wildmax = 0
    
    if games.index(game) == 3:
        if pkmn in [row[0] for row in tradesyellow]:
            trademax = 100
        else:
            trademax = 0
        if pkmn in [row[0] for row in giftsyellow]:
            giftlevel = int(giftsyellow[[row[0] for row in giftsyellow].index(pkmn)][1])
        else:
            giftlevel = 0
    else:
        if pkmn in [row[0] for row in tradesrb]:
            trademax = 100
        else:
            trademax = 0
        if pkmn in [row[0] for row in giftsrb]:
            giftlevel = int(giftsrb[[row[0] for row in giftsrb].index(pkmn)][1])
        else:
            giftlevel = 0
    maxlevel = max([wildmax,trademax,giftlevel])
    
    if maxlevel == 0:
        return ''
    else:
        return maxlevel
    
def get_min_level(pkmn,game):
    try:
        wildmin = min([int(spot[-2]) for spot in locationstats[pkmn][game] if spot[-2].isdigit()])
    except:
        wildmin = 101
    
    if games.index(game) == 3:
        if pkmn in [row[0] for row in tradesyellow]:
            trademin = trademinlevelsyellow[pkmn]
        else:
            trademin = 101
        if pkmn in [row[0] for row in giftsyellow]:
            giftlevel = int(giftsyellow[[row[0] for row in giftsyellow].index(pkmn)][1])
        else:
            giftlevel = 101
    else:
        if pkmn in [row[0] for row in tradesrb]:
            trademin = trademinlevelsrb[pkmn]
        else:
            trademin = 101
        if pkmn in [row[0] for row in giftsrb]:
            giftlevel = int(giftsrb[[row[0] for row in giftsrb].index(pkmn)][1])
        else:
            giftlevel = 101
    minlevel = min([wildmin,trademin,giftlevel])
    
    if minlevel == 101:
        return ''
    else:
        return minlevel

maxlevels = {key: [get_max_level(key,game) for game in games] for key in dex}
minlevels = {key: [get_min_level(key,game) for game in games] for key in dex}

In [536]:
def calc_max_stats(key,game):
    if get_max_level(key,game)=='':
        return ''
    else:
        level = maxlevels[key][games.index(game)]
        hp = math.floor((stats[key][1]+15)*2*level/100.0)+level+10
        other = [math.floor((stat+15)*2*level/100.0)+5 for stat in stats[key][2:]]
        return hp+sum(other)
    
def calc_min_stats(key,game):
    if get_min_level(key,game)=='':
        return ''
    else:
        level = minlevels[key][games.index(game)]
        hp = math.floor(stats[key][1]*2*level/100.0)+level+10
        other = [math.floor(stat*2*level/100.0+5)for stat in stats[key][2:]]
        return hp+sum(other)
    
maxstattotals = {game: {key: calc_max_stats(key,game) for key in dex} for game in games}
minstattotals = {game: {key: calc_min_stats(key,game) for key in dex} for game in games}

def return_all_stats(key,game):
    return [int(key)] + [dex[key]] + stats[key] + [minlevels[key][games.index(game)]] + [minstattotals[game][key]] + [minlevels[key][games.index(game)]*stats[key][0]] + [maxlevels[key][games.index(game)]] + [maxstattotals[game][key]] + [maxlevels[key][games.index(game)]*stats[key][0]]

all_stats = {game: [return_all_stats(key,game) for key in dex] for game in games}

In [537]:
with open("pkmnred.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(all_stats[games[0]])

In [538]:
with open("pkmnyellow.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(all_stats[games[3]])

## Gen II

### Import data

In [543]:
if(not(os.path.isdir('gen_ii_htmls'))):
    os.mkdir('gen_ii_htmls')

In [553]:
url = "https://www.serebii.net/pokedex-gs/"

urllib.request.urlretrieve(url, "gen_ii_htmls/gen_ii.txt")

outputFile = codecs.open("gen_ii_htmls/gen_ii.txt", "r", "ISO-8859-1")
filestring = outputFile.read()

dex = dict(re.findall("(?s)<option value=\"/pokedex-gs/(.*?).shtml\">(.*?)</option>", filestring, re.IGNORECASE))

In [554]:
# Gold and Silver Trades
url = 'https://www.serebii.net/gs/trades.shtml'
filename = 'gen_ii_htmls/tradesgs.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
tradesourcegs = outputFile.read()


# Yellow Trades
url = 'https://www.serebii.net/crystal/trades.shtml'
filename = 'gen_ii_htmls/tradescrystal.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
tradesourcecrystal = outputFile.read()


# Red and Blue Gifts
url = 'https://www.serebii.net/gs/gift.shtml'
filename = 'gen_ii_htmls/giftsgs.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
giftsourcegs = outputFile.read()

# Yellow Gifts
url = 'https://www.serebii.net/crystal/gift.shtml'
filename = 'gen_ii_htmls/giftscrystal.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
giftsourcecrystal = outputFile.read()

In [556]:
info = {}

for key in dex:
    url = "https://www.serebii.net/pokedex-gs/" + key + ".shtml"
    file = "gen_ii_htmls/" + dex[key] + ".txt"
    
    urllib.request.urlretrieve(url, file)
    
    outputFile = codecs.open(file, "r", "ISO-8859-1")
    info[key] = outputFile.read()

In [611]:
locations = {}

for key in dex:
    url = "https://www.serebii.net/pokedex-gs/location/" + key + ".shtml#r"
    file = "gen_ii_htmls/" + dex[key] + "_loc" +".txt"
    
    urllib.request.urlretrieve(url, file)
    
    outputFile = codecs.open(file, "r", "ISO-8859-1")
    locations[key] = outputFile.read()

### Parse Data

In [562]:
searchstring = '(?s)Base Stats - Total: (.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td>\r\n<td align="center" class="fooinfo">(.*?)</td></tr>\r\n'
stats = {}

for key in dex:
    stats[key] = [int(num) for num in re.findall(searchstring, info[key], re.IGNORECASE)[0]]

In [573]:
searchstring = '(?s)<img src="/pokearth/sprites/.*?/(.*?).png" border="0" title="" />.*?Trader requests a.*?<a href="/pokedex-.*?/(.*?).shtml">'
tradesgs = re.findall(searchstring,tradesourcegs,re.IGNORECASE)

searchstring = '(?s)<img src="/pokearth/sprites/.*?/(.*?).png" border="0" title="" />.*?Trader requests a.*?<a href="/pokedex-.*?/(.*?).shtml">'
tradescrystal = re.findall(searchstring,tradesourcecrystal,re.IGNORECASE)

In [598]:
searchstring = '(?s)<td class="label">.*?</td></tr><tr><td class="pkmn"><a href="/pokedex-gs/(.*?).shtml"><img  src="/pokearth/sprites/gold/.*?.png" border="0" title=".*?" /></a></td></tr><tr>\r\n\t\t<td class="label">Level (.*?)</td>'
giftsgs = re.findall(searchstring,giftsourcegs,re.IGNORECASE)

searchstring = '(?s)<td class="label">.*?</td></tr><tr><td class="pkmn"><a href="/pokedex-gs/(.*?).shtml"><img  src="/pokearth/sprites/crystal/.*?.png" border="0" title=".*?" /></a></td></tr><tr>\r\n\t\t<td class="label">Level (.*?)</td>'
giftscrystal = re.findall(searchstring,giftsourcecrystal,re.IGNORECASE)

In [637]:
searchstring = '(?s)<table class="dextable"><tr > <td class="fooevo" colspan="6"><a name=".*?">(.*?)</a></td></tr>(.*?)</table>\r\n'

subsearchstring = '(?s)\t\t<a href="/pokearth/.*?/2nd/.*?.shtml">(.*?)</a>\t\t</td>\r\n\t\t<td class="fooinfo">(.*?)\t\t</td><td class="fooinfo">(.*?)\t</td>\r\n\t\t<td class="fooinfo">(.*?)\t\t</td>\r\n\t\t<td class="fooinfo">(.*?)</td>\r\n\t\t<td class="fooinfo">(.*?)</td>\r\n'

locationstats = {}

for key in dex:
    locationstats[key] = {game[0]: re.findall(subsearchstring,game[1],re.IGNORECASE) for game in re.findall(searchstring, locations[key], re.IGNORECASE)}

In [638]:
games = [key for key in locationstats['010']]

In [654]:
trademinlevelsgs = {'095': 3,\
                    '066': 5,\
                    '100': 5,\
                    '112': 30,\
                    '078': 14,\
                    '142': 5}

trademinlevelscrystal = {'095': 3,\
                         '066': 5,\
                         '100': 5,\
                         '085': 30,\
                         '178': 15,\
                         '082': 16,\
                         '142': 5}

def get_max_level(pkmn,game):
    try:
        wildmax = max([int(spot[-1]) for spot in locationstats[pkmn][game] if spot[-1].isdigit()])
    except:
        wildmax = 0
    
    if games.index(game) == 2:
        if pkmn in [row[0] for row in tradescrystal]:
            trademax = 100
        else:
            trademax = 0
        if pkmn in [row[0] for row in giftscrystal]:
            giftlevel = int(giftscrystal[[row[0] for row in giftscrystal].index(pkmn)][1])
        else:
            giftlevel = 0
    else:
        if pkmn in [row[0] for row in tradesgs]:
            trademax = 100
        else:
            trademax = 0
        if pkmn in [row[0] for row in giftsgs]:
            giftlevel = int(giftsgs[[row[0] for row in giftsgs].index(pkmn)][1])
        else:
            giftlevel = 0
    maxlevel = max([wildmax,trademax,giftlevel])
    
    if maxlevel == 0:
        return ''
    else:
        return maxlevel
    
def get_min_level(pkmn,game):
    try:
        wildmin = min([int(spot[-2]) for spot in locationstats[pkmn][game] if spot[-2].isdigit()])
    except:
        wildmin = 101
    
    if games.index(game) == 2:
        if pkmn in [row[0] for row in tradescrystal]:
            trademin = trademinlevelscrystal[pkmn]
        else:
            trademin = 101
        if pkmn in [row[0] for row in giftscrystal]:
            giftlevel = int(giftscrystal[[row[0] for row in giftscrystal].index(pkmn)][1])
        else:
            giftlevel = 101
    else:
        if pkmn in [row[0] for row in tradesgs]:
            trademin = trademinlevelsgs[pkmn]
        else:
            trademin = 101
        if pkmn in [row[0] for row in giftsgs]:
            giftlevel = int(giftsgs[[row[0] for row in giftsgs].index(pkmn)][1])
        else:
            giftlevel = 101
    minlevel = min([wildmin,trademin,giftlevel])
    
    if minlevel == 101:
        return ''
    else:
        return minlevel

maxlevels = {key: [get_max_level(key,game) for game in games] for key in dex}
minlevels = {key: [get_min_level(key,game) for game in games] for key in dex}

In [655]:
def calc_max_stats(key,game):
    if get_max_level(key,game)=='':
        return ''
    else:
        level = maxlevels[key][games.index(game)]
        hp = math.floor((stats[key][1]+15)*2*level/100.0)+level+10
        other = [math.floor((stat+15)*2*level/100.0)+5 for stat in stats[key][2:]]
        return hp+sum(other)
    
def calc_min_stats(key,game):
    if get_min_level(key,game)=='':
        return ''
    else:
        level = minlevels[key][games.index(game)]
        hp = math.floor(stats[key][1]*2*level/100.0)+level+10
        other = [math.floor(stat*2*level/100.0+5)for stat in stats[key][2:]]
        return hp+sum(other)
    
maxstattotals = {game: {key: calc_max_stats(key,game) for key in dex} for game in games}
minstattotals = {game: {key: calc_min_stats(key,game) for key in dex} for game in games}

def return_all_stats(key,game):
    return [int(key)] + [dex[key]] + stats[key] + [minlevels[key][games.index(game)]] + [minstattotals[game][key]] + [minlevels[key][games.index(game)]*stats[key][0]] + [maxlevels[key][games.index(game)]] + [maxstattotals[game][key]] + [maxlevels[key][games.index(game)]*stats[key][0]]

all_stats = {game: [return_all_stats(key,game) for key in dex] for game in games}

In [656]:
with open("pkmngold.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(all_stats[games[0]])

In [657]:
with open("pkmncrystal.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(all_stats[games[2]])

## Gen III

### Import data

In [658]:
if(not(os.path.isdir('gen_iii_htmls'))):
    os.mkdir('gen_iii_htmls')

In [670]:
url = "https://www.serebii.net/pokedex-rs/"

urllib.request.urlretrieve(url, "gen_iii_htmls/gen_iii.txt")

outputFile = codecs.open("gen_iii_htmls/gen_iii.txt", "r", "ISO-8859-1")
filestring = outputFile.read()

dex = dict(re.findall("(?s)<option value=\"/pokedex-rs/(.*?).shtml\">[0-9][0-9][0-9] (.*?)\r\r\n", filestring, re.IGNORECASE))

In [745]:
# Ruby and Sapphire Trades
url = 'https://www.serebii.net/rubysapphire/trade.shtml'
filename = 'gen_iii_htmls/tradesrs.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
tradesourcers = outputFile.read()


# Emerald Trades
url = 'https://www.serebii.net/emerald/trade.shtml'
filename = 'gen_iii_htmls/tradesemerald.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
tradesourceemerald = outputFile.read()


# Ruby, Sapphire, Emerald Gifts
url = 'https://www.serebii.net/rubysapphire/gift.shtml'
filename = 'gen_iii_htmls/giftsrse.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
giftsourcerse = outputFile.read()

# FireRed and LeafGreen Trades
url = 'https://www.serebii.net/fireredleafgreen/trades.shtml'
filename = 'gen_iii_htmls/tradesfrlg.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
tradesourcefrlg = outputFile.read()

# FireRed and LeafGreen Gifts
url = 'https://www.serebii.net/fireredleafgreen/gift.shtml'
filename = 'gen_iii_htmls/giftsfrlg.txt'

urllib.request.urlretrieve(url, filename)

outputFile = codecs.open(filename, "r", "ISO-8859-1")
giftsourcefrlg = outputFile.read()

In [673]:
info = {}

for key in dex:
    url = "https://www.serebii.net/pokedex-rs/" + key + ".shtml"
    file = "gen_iii_htmls/" + dex[key] + ".txt"
    
    urllib.request.urlretrieve(url, file)
    
    outputFile = codecs.open(file, "r", "ISO-8859-1")
    info[key] = outputFile.read()

In [675]:
locations = {}

for key in dex:
    url = "https://www.serebii.net/pokedex-rs/location/" + key + ".shtml#r"
    file = "gen_iii_htmls/" + dex[key] + "_loc" +".txt"
    
    urllib.request.urlretrieve(url, file)
    
    outputFile = codecs.open(file, "r", "ISO-8859-1")
    locations[key] = outputFile.read()

### Parse Data

In [679]:
searchstring = '(?s)<td>Base Stats</td><td align="center">(.*?)</td><td align="center">(.*?)</td><td align="center">(.*?)</td><td align="center">(.*?)</td><td align="center">(.*?)</td><td align="center">(.*?)</td>'
stats = {}

for key in dex:
    statsind = [int(num) for num in re.findall(searchstring, info[key], re.IGNORECASE)[0]]
    stats[key] = [sum(statsind)] + statsind

In [752]:
searchstring = '(?s)<img src="/pokearth/sprites/.*?/(.*?).png" border="0" title="" />.*?Trader requests a.*?<a href="/pokedex-.*?/(.*?).shtml">'
tradesrs = re.findall(searchstring,tradesourcers,re.IGNORECASE)

searchstring = '(?s)<a href="/pokedex-rs/dex.php\?id=(.*?)"><img src="/emerald/pokemon/.*?.png" border=></a>.*?<img src="/emerald/items/.*?.gif" align="left"><a href="/pokedex-rs/dex.php\?id=.*?">#(.*?) .*?</a><br>'
tradesemerald = [(row[1],row[0]) for row in re.findall(searchstring,tradesourceemerald,re.IGNORECASE)]

searchstring = '(?s)<img src="/pokearth/sprites/.*?/(.*?).png" border="0" title="" />.*?Trader requests a.*?<a href="/pokedex-.*?/(.*?).shtml">'
tradesfrlg = re.findall(searchstring,tradesourcefrlg,re.IGNORECASE)

In [753]:
searchstring = '(?s)<td class="label">.*?</td></tr><tr><td class="pkmn"><a href="/pokedex-rs/(.*?).shtml"><img  src="/pokearth/sprites/.*?/.*?.png" border="0" title=".*?" /></a></td></tr><tr>\r\n\t\t<td class="label">Level (.*?)</td>'
giftsrse = re.findall(searchstring,giftsourcerse,re.IGNORECASE)

searchstring = '(?s)<td class="label">.*?</td></tr><tr><td class="pkmn"><a href="/pokedex-rs/(.*?).shtml"><img  src="/pokearth/sprites/.*?/.*?.png" border="0" title=".*?" /></a></td></tr><tr>\r\n\t\t<td class="label">Level (.*?)</td>'
giftsfrlg = re.findall(searchstring,giftsourcefrlg,re.IGNORECASE)

In [730]:
searchstring = '(?s)<table class="dextable"><tr > <td class="fooevo" colspan="6"><a name=".*?">(.*?)</a></td></tr>\n(.*?)</table>'

subsearchstring = '(?s)\t\t<td align="center" class="fooinfo"><a href="/pokearth/.*?/.*?.shtml">(.*?)</a><td align="center" class="fooinfo">(.*?)</td>\n<td align="center" class="fooinfo">(.*?)</td>\n<td align="center" class="fooinfo">(.*?)</td>\n<td align="center" class="fooinfo">(.*?)</td>'

locationstats = {}

for key in dex:
    locationstats[key] = {game[0]: re.findall(subsearchstring,game[1],re.IGNORECASE) for game in re.findall(searchstring, locations[key], re.IGNORECASE)}

In [754]:
games = [key for key in locationstats['129']]

In [759]:
trademinlevelsrs = {'296': 5,\
                    '300': 3,\
                    '222': 21}

trademinlevelsemerald = {'311': 12,\
                         '116': 5,\
                         '052': 5}

trademinlevelsfrlg = {'122': 5,\
                      '029': 5,\
                      '032': 5,\
                      '030': 16,\
                      '033': 16,\
                      '108': 25,\
                      '124': 20,\
                      '083': 3,\
                      '086': 5,\
                      '101': 3,\
                      '114': 5}

def get_max_level(pkmn,game):
    try:
        wildmax = max([int(spot[-1]) for spot in locationstats[pkmn][game] if spot[-1].isdigit()])
    except:
        wildmax = 0
    
    if games.index(game) < 2:
        if pkmn in [row[0] for row in tradesrs]:
            trademax = 100
        else:
            trademax = 0
            
        if pkmn in [row[0] for row in giftsrse]:
            giftlevel = int(giftsrse[[row[0] for row in giftsrse].index(pkmn)][1])
        else:
            giftlevel = 0
    elif games.index(game) == 2:
        if pkmn in [row[0] for row in tradesemerald]:
            trademax = 100
        else:
            trademax = 0
            
        if pkmn in [row[0] for row in giftsrse]:
            giftlevel = int(giftsrse[[row[0] for row in giftsrse].index(pkmn)][1])
        else:
            giftlevel = 0
    else:
        if pkmn in [row[0] for row in tradesfrlg]:
            trademax = 100
        else:
            trademax = 0
            
        if pkmn in [row[0] for row in giftsfrlg]:
            giftlevel = int(giftsfrlg[[row[0] for row in giftsfrlg].index(pkmn)][1])
        else:
            giftlevel = 0
    maxlevel = max([wildmax,trademax,giftlevel])
    
    if maxlevel == 0:
        return ''
    else:
        return maxlevel
    
def get_min_level(pkmn,game):
    try:
        wildmin = min([int(spot[-2]) for spot in locationstats[pkmn][game] if spot[-2].isdigit()])
    except:
        wildmin = 101
    
    if games.index(game) < 2:
        if pkmn in [row[0] for row in tradesrs]:
            trademin = trademinlevelsrs[pkmn]
        else:
            trademin = 101
        if pkmn in [row[0] for row in giftsrse]:
            giftlevel = int(giftsrse[[row[0] for row in giftsrse].index(pkmn)][1])
        else:
            giftlevel = 101
    elif games.index(game) == 2:
        if pkmn in [row[0] for row in tradesemerald]:
            trademin = trademinlevelsemerald[pkmn]
        else:
            trademin = 101
        if pkmn in [row[0] for row in giftsrse]:
            giftlevel = int(giftsrse[[row[0] for row in giftsrse].index(pkmn)][1])
        else:
            giftlevel = 101
    else:
        if pkmn in [row[0] for row in tradesfrlg]:
            trademin = trademinlevelsfrlg[pkmn]
        else:
            trademin = 101
        if pkmn in [row[0] for row in giftsfrlg]:
            giftlevel = int(giftsfrlg[[row[0] for row in giftsfrlg].index(pkmn)][1])
        else:
            giftlevel = 101
    minlevel = min([wildmin,trademin,giftlevel])
    
    if minlevel == 101:
        return ''
    else:
        return minlevel

maxlevels = {key: [get_max_level(key,game) for game in games] for key in dex}
minlevels = {key: [get_min_level(key,game) for game in games] for key in dex}

In [772]:
def calc_max_stats(key,game):
    if get_max_level(key,game)=='':
        return ''
    else:
        level = maxlevels[key][games.index(game)]
        hp = math.floor((stats[key][1]*2+31)*level/100.0)+level+10
        nonhpstats = stats[key][2:]
        nature = [1.0 for stat in nonhpstats]
        minstat = nonhpstats.index(min(nonhpstats))
        maxstat = nonhpstats.index(max(nonhpstats))
        nature[maxstat] = 1.1
        nature[minstat] = 0.9
        other = [math.floor((math.floor((nonhpstats[i]*2+31)*level/100.0)+5)*nature[i]) for i in range(len(nonhpstats))]
        return hp+sum(other)
    
def calc_min_stats(key,game):
    if get_min_level(key,game)=='':
        return ''
    else:
        level = minlevels[key][games.index(game)]
        hp = math.floor((stats[key][1]*2)*level/100.0)+level+10
        nonhpstats = stats[key][2:]
        nature = [1.0 for stat in nonhpstats]
        minstat = nonhpstats.index(min(nonhpstats))
        maxstat = nonhpstats.index(max(nonhpstats))
        nature[minstat] = 1.1
        nature[maxstat] = 0.9
        other = [math.floor((math.floor((nonhpstats[i]*2)*level/100.0)+5)*nature[i]) for i in range(len(nonhpstats))]
        return hp+sum(other)
    
maxstattotals = {game: {key: calc_max_stats(key,game) for key in dex} for game in games}
minstattotals = {game: {key: calc_min_stats(key,game) for key in dex} for game in games}

def return_all_stats(key,game):
    return [int(key)] + [dex[key]] + stats[key] + [minlevels[key][games.index(game)]] + [minstattotals[game][key]] + [minlevels[key][games.index(game)]*stats[key][0]] + [maxlevels[key][games.index(game)]] + [maxstattotals[game][key]] + [maxlevels[key][games.index(game)]*stats[key][0]]

all_stats = {game: [return_all_stats(key,game) for key in dex] for game in games}

In [773]:
with open("pkmnruby.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(all_stats[games[0]])

In [774]:
with open("pkmnemerald.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(all_stats[games[2]])

In [775]:
with open("pkmnfirered.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(all_stats[games[3]])